<a href="https://colab.research.google.com/github/951237/TIL/blob/master/%EC%9B%B9%EC%95%B1_%EB%82%A0%EC%94%A8%EC%98%88%EC%B8%A1_%ED%94%84%EB%A1%9C%EA%B7%B8%EB%9E%A8_200503.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## TIL - 200210 : 배추가격 예측 AI로봇 
- 링크 : https://youtu.be/wZvhBVqitn8

## TIL - 200209 텐서플로 따라하기(안경잡이 개발자)
- x축은 시간 
- y축은 임금
- 데이터를 기반으로 추후 임금 구하기

### 데이터 수집
- 기상자료개방포털 
    - 기온 : https://data.kma.go.kr/stcs/grnd/grndTaList.do?pgmNo=70
    - 강수량 : https://data.kma.go.kr/stcs/grnd/grndRnList.do?pgmNo=69
- 농산물 유통정보 / 채소류 / 배추 / 전체품종
    - 소매가격 / 최소가격

### 데이터 정제
- 평균기온, 최저기온, 최고기온, 강수량, 가격

### 텐서플로 다변인 선형 모델 계발하기
- 변수가 여러가지 인경우 다변인 선형 회귀모델 적용
    - 식 : H(x1, x2, x3, x4) = x1w1 + x2w2 + x3w3 + x4w4 
        - 모든 변인이 가격에 영향을 미친다고 감안. 가중치를 고려하여 식을 세움.
        - 행렬의 곱을 이요한 식 : H(X) = XW
    - 학습률 : 0.000005

## 텐서플로 2.0 변환 성공

참고 : https://leejigun.github.io/tensorflow2_2

멀티회구 예측 : https://leejigun.github.io/tensorflow2_3

추가작업 : 학습중 결과 보여주기

### 훈련하기

In [0]:

# 라이브러리 호출
import tensorflow as tf
import numpy as np
import pandas as pd

# 데이터 불러오기
data = pd.read_csv('/content/drive/My Drive/data/ML/배추가격_데이터_예제_price data.csv')   # 데이터 파일 불러오기

# 학습데이터 준비하기
xy = np.array(data, dtype=np.float32)       # 데이터를 넘파이 값으로 담기 
x_data = xy[:, 1:-1]    # 모든행을 선택, 열은 2번째 부터 끝에서 2번째 열까지 선택(변수 4가지)
y_data = xy[:, [-1]]    # 모든행, 마지막 열 선택(가격)

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(1, input_dim = 4))
sgd = tf.keras.optimizers.SGD(learning_rate=0.000005)
model.compile(loss='mean_squared_error', optimizer=sgd)
model.fit(x_data, y_data, epochs = 2000, batch_size=100)

### 결과 예측하기

In [12]:
data = ((23.2, 20.2, 27.3, 7.4), )
arr = np.array(data, dtype=np.float32)

xdata = arr[0:4]
xdata
print(model.predict(xdata))

[[3072.3857]]


## 배추값 예측 코드 v1

In [0]:

# 라이브러리 호출
import tensorflow as tf
import numpy as np
import pandas as pd

# 데이터 불러오기
data = pd.read_csv('/content/drive/My Drive/data/ML/배추가격_데이터_예제_price data.csv')   # 데이터 파일 불러오기

# 학습데이터 준비하기
model = tf.global_variables_initializer()   # 텐서플로 변수 초기화 하기
xy = np.array(data, dtype=np.float32)       # 데이터를 넘파이 값으로 담기 
x_data = xy[:, 1:-1]    # 모든행을 선택, 열은 2번째 부터 끝에서 2번째 열까지 선택(변수 4가지)
y_data = xy[:, [-1]]    # 모든행, 마지막 열 선택(가격)


# x = tf.placeholder(tf.float32, shape=[None, 4])     # x 축에 들어갈 학습데이터 선언
# y = tf.placeholder(tf.float32, shape=[None, 1])     # y 축에 들어갈 학습데이터 선언
W = tf.Variable(tf.random_normal([4,1]), name = 'weight') # 가중치의 변수를 랜덤으로 선언 [4,1]의 의미는 변수의 모양 4개 1행
b = tf.Variable(tf.random_normal([1]), name='bias')       # 바이어스의 변수를 랜덤으로 선언 [1]의 의미는 변수의 형태임 1개 1행

# 가설식 세우기
hypothesis = tf.matmul(x,w) + b     # 가설식 세우기. 행열의 곱을 이용하여 식 세우기

# 계산식 세우기
cost = tf.reduce_mean(tf.square(hypothesis - y))    # 계산식 세우기

# 최적화 함수 선언
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.000005) # 최적화 함수 선언 및 학습율 설정하기
train = optimizer.minimize(cost)

# 세선 만들기
sess = tf.Session() # 세션 만들기
sess.run(tf.global_variables_initializer())   # 변수값 초기화

# 학습시키기
for step in range(100001):
    cost_, hypo_, _ = sess.run([cost, hypothesis, train], feed_dict={x : x_data, y : y_data})
    if step % 500 == 0:
        print('#', step, "손실비용 : ", cost_)
        print("- 배추가격 : ", hypo_[0])

# 학습모델 저저장하기 
saver = tf.train.Saver()
save_path = saver.save(sess, '/content/drive/My Drive/data/ML/배추가격예상하기/saved.cpkt')
print("학습된 모델을 저장햇습니다.")